In [ ]:
import tensorflow as tf

In [ ]:
## linspace exists in tf
x = tf.linspace(-3.0,3.0,100)
print(x)

The name of it is "LinSpace:0". Wherever we see this colon 0, that just means the output of. So the name of this Tensor is saying, the output of LinSpace.
Notice that unlike to numpy no values are printerd here. Instead this is merely a pointer to the ouput of a tf.Operation which has now been added to Tensorflow's default computational graph.

The result of this operation is the tensor that we are returned.

We can inspect the default graph by,

In [ ]:
# Here we are taking a copy of the default graph
g = tf.get_default_graph()

In [ ]:
# any operations or tensor can always be obtained from this graph
[op.name for op in g.get_operations()]


In [ ]:
g.get_collection(name='LinSpace/start')
# TODO read about collections

For each operation we have first the parameters for that operation and then the operation which takes all of the parameters and creates an output for the linspace.

We can think about just like we do in linear algebra,

The space of operations forms a dual space for the vector space

In [ ]:
g.get_tensor_by_name('LinSpace' + ':0')


In [ ]:
# We can always reset the graph by
[op.name for op in g.get_operations()]

In [ ]:
# We're first going to create a session:
sess = tf.Session()

# Now we tell our session to compute anything we've created in the tensorflow graph.
computed_x = sess.run(x)
print(computed_x)

# Alternatively, we could tell the previous Tensor to evaluate itself using this session:
computed_x = x.eval(session=sess)
print(computed_x)

# We can close the session after we're done like so:
sess.close()

In [ ]:
# Tells the session which graph we want to manage
sess = tf.Session(graph=g)
sess.close()

Just create a new graph to be explicit

In [ ]:
sess = tf.InteractiveSession()
x.eval()

In [ ]:
type(x.get_shape())

In [ ]:
sh = x.get_shape()

In [ ]:
sh.as_list()

Now let's have some fun!
Create a 1D Gaussian

In [ ]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:

# The 1 dimensional gaussian takes two parameters, the mean value, and the standard deviation, which is commonly denoted by the name sigma.
mean = 0.0
sigma = 1.0

# Don't worry about trying to learn or remember this formula.  
z = (tf.exp(tf.neg(tf.pow(x - mean, 2.0) /
                   (2.0 * tf.pow(sigma, 2.0)))) *
     (1.0 / (sigma * tf.sqrt(2.0 * np.pi))))

In [ ]:
res = z.eval()

plt.plot(res)

To get a k size Gaussian kernel of two dimension:

In [ ]:
# Rember this is the linspace dimension
z.get_shape().as_list()

In [ ]:
# Let's store the number of values in our Gaussian curve.
ksize = z.get_shape().as_list()[0]

# Let's multiply the two to get a 2d gaussian
z_2d = tf.matmul(tf.reshape(z, [ksize, 1]), tf.reshape(z, [1, ksize]))

# Execute the graph
plt.imshow(z_2d.eval())

We would like to place Gaussians such as this one on body joint locations.

To do this we need first need to obtain a single example from the TFRecords representing the MPII dataset

In [ ]:
import tensorflow as tf

In [ ]:
from input_pipeline import setup_eval_input_pipeline

In [ ]:
sess = tf.InteractiveSession()

In [ ]:
data_dir = 'data/' # Path to tfrecords files
input_queue_memory_factor = 16 # Factor which decides the number of minimum examples in RandomShuffleQueue
batch_size = 32 # Only for demonstration
num_preprocess_threads = 1 # number of preprocessing threads used
image_dim = 380 # Same number of pixels as in Bulat

example=setup_eval_input_pipeline(data_dir,
                           batch_size,
                           num_preprocess_threads,
                           image_dim)

In [ ]:
coord = tf.train.Coordinator()

In [ ]:
threads = tf.train.start_queue_runners(sess=sess, coord=coord)

In [ ]:
sess.run(example.images)

Doesn't work??

In [ ]:
data_filenames = tf.gfile.Glob('*.tfrecord')

In [ ]:
type(data_filenames[0])

In [ ]:
import glob

In [ ]:
glob.glob('./data/*')

In [ ]:
coord = tf.train.Coordinator()
res = example.images.eval()

In [ ]:
example.x_joints
my_first_img = example.images.eval()
print(my_first_img.shape)

In [ ]:
examples_queue = tf.RandomShuffleQueue(capacity=0,min_after_dequeue=0,dtypes=[tf.string])

In [ ]:
import tensorflow as tf

def mpii_read_and_decode_single_example(tfrecord):
    '''
    args: tfrecord for the mpii dataset
    returns: parsed example from the tfrecord
    '''
    
    # first construct a queue containing a list of filenames.
    # this lets a user split up there dataset in multiple files to keep
    # size down
    filename_queue = tf.train.string_input_producer([tfrecord],
                                                    num_epochs=None)
    
    # Unlike the TFRecordWriter, the TFRecordReader is symbolic
    reader = tf.TFRecordReader()
    
    # One can read a single serialized example from a filename
    # serialized_example is a Tensor of type string.
    _, serialized_example = reader.read(filename_queue)
    
    # the feature map for the mpii dataset allows us to convert the serialized
    # example back to non-serialized values
    feature_map = {
        'image_jpeg': tf.FixedLenFeature(shape=[], dtype=tf.string),
        'joint_indices': tf.VarLenFeature(dtype=tf.int64),
        'x_joints': tf.VarLenFeature(dtype=tf.float32),
        'y_joints': tf.VarLenFeature(dtype=tf.float32),
        'head_size': tf.FixedLenFeature(shape=[], dtype=tf.float32)
    }
    
    # The serialized example is converted back to actual values.
    # One needs to describe the format of the objects to be returned
    example = tf.parse_single_example(
        serialized_example,
        features=feature_map)
    # now return the converted data
    return example

In [ ]:
x = mpii_read_and_decode_single_example('data/train0.tfrecord')

In [ ]:
img_jpeg = x['image_jpeg']

In [ ]:
img_tensor = tf.image.decode_jpeg(contents=img_jpeg, channels=3)
print(img_tensor)

In [ ]:
decoded_img = tf.image.convert_image_dtype(image=img_tensor, dtype=tf.float32)

In [ ]:
reshaped_img = tf.reshape(tensor=decoded_img,shape=[380,380,3])

In [ ]:
sess = tf.Session()
sess.run(tf.initialize_all_variables())
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)

In [ ]:
x = sess.run(reshaped_img)

In [ ]:
g=tf.get_default_graph()

In [ ]:
[op.name for op in g.get_operations()]

In [ ]:
sess = tf.Session(graph=g)

In [ ]:
sess.run(tf.initialize_all_variables())

In [ ]:
tf.train.start_queue_runners(sess=sess)

In [ ]:
img_1 = sess.run(decoded_img)

In [ ]:
sess=tf.Session()

#changes in later verisions
init = tf.initialize_all_variables()
sess.run(init)
# What happens here?
tf.train.start_queue_runners(sess=sess)


decoded_img_1 = sess.run([decoded_img])
decoded_img_2 = sess.run([decoded_img])

In [ ]:
# returns symbolic label and image
label, image = read_and_decode_single_example("mnist.tfrecords")

sess = tf.Session()

# Required. See below for explanation
init = tf.initialize_all_variables()
sess.run(init)
tf.train.start_queue_runners(sess=sess)

# grab examples back.
# first example from file
label_val_1, image_val_1 = sess.run([label, image])
# second example from file
label_val_2, image_val_2 = sess.run([label, image])

In [1]:
import mpii_read

In [3]:
mpii_read.